In [ ]:
import pprint
import numpy
import matplotlib.pyplot as pyplot
from sklearn.metrics.pairwise import cosine_similarity
import mss.spectra
import mss.similarity
from pyteomics import fasta
import mss.db
from scipy.sparse import lil_matrix

In [ ]:
%matplotlib inline

# preprocessing

In [ ]:
mgf_file = 'data/spectra/amethyst_annotated.mgf'
spectra = mss.spectra.read_mgf(mgf_file)

In [ ]:
spectrum = spectra[0]
pprint.pprint(spectrum)

In [ ]:
def plot_spectrum(mz, intensity=None):
    intensity = (intensity if intensity is not None
                 else numpy.ones_like(mz))
    pyplot.vlines(mz, 0, intensity)

plot_spectrum(spectrum['m/z array'], spectrum['intensity array'])

In [ ]:
sequence = spectrum['params']['peptide']
print(sequence)
mss.spectra.b_ionts(sequence)

In [ ]:
mz = mss.spectra.compute_mass_spectrum(sequence)
plot_spectrum(mz)

In [ ]:
bins = 10
binned = mss.spectra.bin_spectrum(spectrum['m/z array'], spectrum['intensity array'], bins)
pyplot.vlines(range(0, bins), 0, binned)

## binning range

compute the binning range from test data

so the binning range will be (150, 4021)

In [ ]:
max(max(spectrum['m/z array']) for spectrum in spectra)

In [ ]:
mgf_file_2 = 'data/spectra/opal_annotated.mgf'
spectra_2 = mss.spectra.read_mgf(mgf_file_2)
max(max(spectrum['m/z array']) for spectrum in spectra_2)

In [ ]:
min(min(spectrum['m/z array']) for spectrum in spectra)

In [ ]:
min(min(spectrum['m/z array']) for spectrum in spectra_2)

In [ ]:
for bin_size in [0.1, 0.5, 1]:
    print('bin_size:', bin_size, 'gives number of bins:', int((4021 - 150) / bin_size))

# database

In [ ]:
bins = 3871

In [ ]:
%%time
fasta_file = 'data/sequence_database/amop_msdb_10000.fasta'
peptides, mzs, intensity_matrix = mss.db.generate_db(fasta_file, bins=bins)

In [ ]:
peptides.shape, mzs.shape, intensity_matrix.shape

In [ ]:
db_file = 'data/db.npz'
numpy.savez(db_file, peptides=peptides, mzs=mzs, intensity_matrix=intensity_matrix)

In [ ]:
a, b, c = mss.db.get_db(db_file)

In [ ]:
a.shape, b.shape, c

# similarity

In [ ]:
mz_matrix = mss.similarity.bin_spectra(spectra, bins=bins)

In [ ]:
mz_matrix.shape, intensity_matrix.shape

In [ ]:
%timeit result = mss.similarity.knn_query(mz_matrix, intensity_matrix, 10)

In [ ]:
result.shape

## top 10 accuracy

In [ ]:
tp = 0
for i in range(result.shape[0]):
    peptide = spectra[i]['params']['peptide']
    if peptide in peptides[result[i]]:
        tp += 1

(tp / result.shape[0]) * 100

## top 5 accuracy

In [ ]:
tp = 0
for i in range(result.shape[0]):
    peptide = spectra[i]['params']['peptide']
    if peptide in peptides[result[i, -5:]]:
        tp += 1

(tp / result.shape[0]) * 100

## accuracy

In [ ]:
tp = 0
for i in range(result.shape[0]):
    peptide = spectra[i]['params']['peptide']
    if peptide in peptides[result[i, -1]]:
        tp += 1

(tp / result.shape[0]) * 100

## range query

In [ ]:
def range_query(spectra_mat, database_mat, treshold):
    n_spectra = spectra_mat.shape[0]
    similarities = cosine_similarity(mz_matrix, intensities, dense_output=False)
    result = list()
    for i in range(n_spectra):
        row = similarities.getrow(i).toarray()[0]
        sorted_index = numpy.argsort(row)
        result.append(sorted_index[-numpy.sum(row >= treshold):])
    return result